# Maximum likelihood estimate of A00 divergence time using Ust'-Ishim

Using method described by Mendez et al. 2016:

> Using the known age of the Ust’-Ishim individual and the constrained optimization procedure described in Rasmussen et al., we obtained parametric bootstrap estimates for $T_{AR}$ as well as for the mutation rate and the TMRCA of haplogroup K-M526 (Appendix A).

> Briefly, we sampled from the process that generated the observed tree (Figure S2) by simulating the number of single nucleotide variants (SNVs) on each branch as a Poisson draw with mean equal to the observed number of mutations. To obtain bootstrap samples of the three parameters, we maximized their joint likelihood for each tree replicate.

From Rasmussen et al 2014:

> [...] we implemented a Poisson process model for mutations on the tree and used the `constrOptim()` function in R to compute a maximum likelihood TMRCA estimate of 16.9 ky. We then repeated this for 100,000 bootstrap simulations to yield a 95% confidence interval.

In [22]:
import numpy as np

In [149]:
import pandas as pd

In [24]:
from scipy import stats
from scipy import optimize

In [25]:
# total length of the analyzed segment (bp)
LENGTH = 7830000

# known age of Ust'-Ishim (years ago)
UST_ISHIM_AGE = 45000

# counts of mutations on branches observed from the data
OBSERVED_COUNTS = {'a' : 1434, 'd' : 305, 'e' : 19, 'f' : 1591}

In [26]:
def sample_mutations(branch):
    '''Sample mutations on a given branch from a Poisson distribution
    with a mean equal to the observed count on that branch.
    '''
    return stats.poisson.rvs(mu=OBSERVED_COUNTS[branch])

In [27]:
def branch_loglikelihood(mutation_rate, branch_time, sampled_count):
    '''Log-probability of observing a specified number of mutations on
    a branch of a tree given the mutation rate and time.
    '''
    return stats.poisson.logpmf(k=sampled_count, mu=int(mutation_rate * branch_time * LENGTH))

In [165]:
def optim_func(x, sampled_counts):
    '''Function that needs to be optimized.
    
    This is just a sum of the four log-likelihood functions, each for one
    of the four branches in a tree.
    '''
    mutation_rate, branch_time_a, branch_time_d, branch_time_f = x
    
    return -(branch_loglikelihood(mutation_rate, branch_time_a, sampled_counts['a']) + \
             branch_loglikelihood(mutation_rate, branch_time_d, sampled_counts['d']) + \
             branch_loglikelihood(mutation_rate, UST_ISHIM_AGE, sampled_counts['d'] - sampled_counts['e']) + \
             branch_loglikelihood(mutation_rate, branch_time_f, sampled_counts['f']))

In [155]:
def direct_estimation(mutation_counts):
    '''Calculate the estimates of the mutation rate and times of all branches
    by simple cross-multiplication.
    '''
    x = (mutation_counts['d'] - OBSERVED_COUNTS['e']) / UST_ISHIM_AGE
    
    return pd.Series([x / LENGTH] + [mutation_counts[branch] / x for branch in 'adf'],
                     index=['mutation_rate', 'a', 'd', 'f'])

In [50]:
def print_optim_result(res):
    '''Print result of the optimization procedure (one row of a dataframe).'''
    fmt_str = '\u00B5: {:.5}\ta: {:.1f}\td: {:.1f}\tf: {:.1f}\t(a+d): {:.1f} \tmean(a+d, f): {:.1f}'
    print(fmt_str.format(res.mutation_rate,
                         res.a,
                         res.d,
                         res.f,
                         res.a + res.d,
                         np.mean([res.a + res.d, res.f])))

In [51]:
def ci_95(x):
    return stats.norm.interval(0.95, loc=np.mean(x), scale=np.std(x)/np.sqrt(len(x)))

# Generating trees by modeling mutation counts from a Poisson distribution
The time on the branch leading to Ust'-Ishim is not estimated. Instead, it is expressed as time leading to the human reference - 45,000 years ago.

In [67]:
num_replicates = 10000

### Initial conditions for the optimization algorithm:

In [68]:
x0 = (
    7e-5,  # mutation rate
    100000, # time on branch a
    100000, # time on branch d
    100000  # time on branch f
)

In [69]:
# dataframe to accumulate results of sampling in
mle_results = pd.DataFrame(columns=['mutation_rate', 'a', 'd', 'f'],
                          index=range(num_replicates),
                          dtype=float)

In [70]:
for i in range(num_replicates):
    sampled_counts = {branch : sample_mutations(branch) for branch in 'adef'}
    mle_results.iloc[i] = optimize.minimize(optim_func1, x0, args=sampled_counts,
                                            method='Nelder-Mead').x
    # print_optim_result(results.iloc[i])

In [71]:
mle_results['a+d'] = mle_results['a'] + mle_results['d']

In [72]:
mle_results.describe()

,mutation_rate,a,d,f,a+d
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.024810e-09,133612.054090,103949.346780,120967.646317,237561.400870
std,1.808750e-11,684.032017,3.363692,684.011205,684.007951
min,9.529377e-10,120929.635605,103942.575369,120929.309131,224873.775408
25%,1.015960e-09,133640.605200,103946.358052,120930.233508,237592.434582
50%,1.024686e-09,133650.854118,103948.694881,120930.351307,237599.396957
75%,1.033247e-09,133659.025314,103951.594908,120930.736803,237605.545313
max,1.093423e-09,133673.222641,103959.087455,133670.974999,237615.798009


### Initial conditions for the optimization algorithm:

In [78]:
x0 = (
    7.8e-10,   # mutation rate
    100000, # time on branch a
    100000, # time on branch d
    100000  # time on branch f
)

In [79]:
# dataframe to accumulate results of sampling in
mle_results = pd.DataFrame(columns=['mutation_rate', 'a', 'd', 'f'],
                           index=range(num_replicates),
                           dtype=float)

In [80]:
for i in range(num_replicates):
    sampled_counts = {branch : sample_mutations(branch) for branch in 'adef'}
    mle_results.iloc[i] = optimize.minimize(optim_func1, x0, args=sampled_counts,
                                            method='Nelder-Mead').x
    # print_optim_result(results.iloc[i])

In [81]:
mle_results['a+d'] = mle_results['a'] + mle_results['d']

In [82]:
mle_results.describe()

,mutation_rate,a,d,f,a+d
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.196351e-09,157698.653380,46658.267649,163066.737894,204356.921029
std,8.634518e-11,15878.901719,550.225229,18701.663389,16268.369357
min,6.536203e-10,120419.360414,45423.218251,134062.693590,167140.739471
25%,1.188600e-09,150175.007877,46353.585574,156922.667377,196629.817454
50%,1.208811e-09,155131.642320,46547.815800,159354.103337,201706.246694
75%,1.231468e-09,160209.583796,46885.230339,161833.213275,206870.822515
max,1.432944e-09,285401.691139,51269.614505,299019.871787,336671.305644


### Initial conditions for the optimization algorithm:

In [73]:
x0 = (
    7e-5,  # mutation rate
    100000, # time on branch a
    100000, # time on branch d
    100000  # time on branch f
)

In [74]:
# dataframe to accumulate results of sampling in
mle_results = pd.DataFrame(columns=['mutation_rate', 'a', 'd', 'f'],
                           index=range(num_replicates),
                           dtype=float)

In [75]:
for i in range(num_replicates):
    sampled_counts = {branch : sample_mutations(branch) for branch in 'adef'}
    mle_results.iloc[i] = optimize.minimize(optim_func, x0, args=sampled_counts,
                                            method='Nelder-Mead').x
    # print_optim_result(results.iloc[i])

In [76]:
mle_results['a+d'] = mle_results['a'] + mle_results['d']

In [77]:
mle_results.describe()

,mutation_rate,a,d,f,a+d
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.144602e-09,133615.920383,103950.074009,120952.352750,237565.994392
std,2.161650e-11,553.691483,2.684890,553.310926,553.580119
min,1.063822e-09,120927.358622,103934.858369,120927.217143,224874.426244
25%,1.131495e-09,133632.807942,103948.813628,120927.758017,237584.840336
50%,1.145002e-09,133638.291030,103950.648775,120928.074945,237589.037348
75%,1.159657e-09,133644.836217,103952.032395,120928.516583,237593.649256
max,1.235816e-09,133694.414492,103955.674846,133651.624638,237629.272861


### Initial conditions for the optimization algorithm:

In [83]:
x0 = (
    7.8e-10,   # mutation rate
    100000, # time on branch a
    100000, # time on branch d
    100000  # time on branch f
)

In [84]:
# dataframe to accumulate results of sampling in
mle_results = DataFrame(columns=['mutation_rate', 'a', 'd', 'f'],
                        index=range(num_replicates),
                        dtype=float)

In [85]:
for i in range(num_replicates):
    sampled_counts = {branch : sample_mutations(branch) for branch in 'adef'}
    mle_results.iloc[i] = optimize.minimize(optim_func, x0, args=sampled_counts,
                                            method='Nelder-Mead').x
    # print_optim_result(results.iloc[i])

In [86]:
mle_results['a+d'] = mle_results['a'] + mle_results['d']

In [90]:
mle_results.describe(percentiles=[])

,mutation_rate,a,d,f,a+d
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,8.128490e-10,226385.369699,47986.979620,251135.615604,274372.349319
std,5.318047e-11,15844.012245,1039.900221,17282.954313,16343.609478
min,6.314286e-10,113142.767739,21651.861321,130048.034541,134794.629060
50%,8.116688e-10,225784.848911,47955.585081,250470.507409,273729.482092
max,1.416153e-09,302396.416621,51473.873611,332056.285984,352054.033677


# Counting using stupid cross-multiplication 'method'

In [22]:
num_replicates = 10000

In [23]:
# dataframe to accumulate results of sampling in
direct_results = pd.DataFrame(columns=['mutation_rate', 'a', 'd', 'f'],
                              index=range(num_replicates),
                              dtype=float)

In [24]:
for i in range(num_replicates):
    sampled_counts = {branch : sample_mutations(branch) for branch in 'adef'}
    direct_results.iloc[i] = direct_estimation(sampled_counts)

In [25]:
direct_results['a+d'] = direct_results['a'] + direct_results['d']

In [26]:
direct_results.describe()

,mutation_rate,a,d,f,a+d
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,8.112513e-10,226547.281960,48002.113373,251402.574098,274549.395334
std,4.897956e-11,15028.002233,182.461663,16526.710615,15195.173404
min,6.101887e-10,176109.510086,47422.096317,196699.716714,223573.487032
25%,7.776359e-10,216151.906780,47878.787879,239657.136658,264027.089415
50%,8.116929e-10,225929.434767,47989.510490,250648.464164,273882.101216
75%,8.429119e-10,236372.747037,48120.437956,262087.912088,284494.382022
max,1.001845e-09,293860.465116,48976.744186,336767.441860,342837.209302


In [27]:
direct_estimation(OBSERVED_COUNTS)

[8.116929189726124e-10,
 225629.37062937065,
 47989.51048951049,
 250332.16783216785]

# Estimations based on original counts

In [216]:
OBSERVED_COUNTS

{'a': 1434, 'd': 305, 'e': 19, 'f': 1591}

In [237]:
res = optimize.minimize(optim_func, (2e-10, 100000, 100000, 100000), args=OBSERVED_COUNTS, method='Nelder-Mead')
print(res)
LENGTH * res.x[0] * res.x[1:]

 final_simplex: (array([[  8.13183106e-10,   2.25299226e+05,   4.79237806e+04,
          2.49912609e+05],
       [  8.13183106e-10,   2.25299226e+05,   4.79237807e+04,
          2.49912609e+05],
       [  8.13183106e-10,   2.25299226e+05,   4.79237806e+04,
          2.49912609e+05],
       [  8.13183106e-10,   2.25299226e+05,   4.79237807e+04,
          2.49912609e+05],
       [  8.13183106e-10,   2.25299226e+05,   4.79237806e+04,
          2.49912609e+05]]), array([ 16.68475154,  16.68475154,  16.68475154,  16.68475154,  16.68475154]))
           fun: 16.684751540865591
       message: 'Optimization terminated successfully.'
          nfev: 420
           nit: 188
        status: 0
       success: True
             x: array([  8.13183106e-10,   2.25299226e+05,   4.79237806e+04,
         2.49912609e+05])


array([ 1434.53057874,   305.1414329 ,  1591.24948984])

In [247]:
res = optimize.differential_evolution(optim_func, bounds=[(1e-11, 9e-9), (100, 500000), (100, 500000), (100, 500000)],
                                      args=(OBSERVED_COUNTS, ), tol=1e-10)
print(res)
LENGTH * res.x[0] * res.x[1:]

     fun: 16.684751540865591
 message: 'Optimization terminated successfully.'
    nfev: 3785
     nit: 62
 success: True
       x: array([  8.13489847e-10,   2.25219191e+05,   4.79476587e+04,
         2.49913221e+05])


array([ 1434.56190029,   305.40862936,  1591.85362665])

In [248]:
res = optimize.differential_evolution(optim_func, bounds=[(1e-11, 9e-9), (100, 500000), (100, 500000), (100, 500000)],
                                      args=(OBSERVED_COUNTS, ))
print(res)
LENGTH * res.x[0] * res.x[1:]

     fun: 16.713742690063327
 message: 'Optimization terminated successfully.'
    nfev: 1925
     nit: 31
 success: True
       x: array([  8.18315735e-10,   2.23689581e+05,   4.78394620e+04,
         2.49710372e+05])


array([ 1433.27135304,   306.52715264,  1599.99728791])

In [224]:
direct_estimation(OBSERVED_COUNTS)

mutation_rate    8.116929e-10
a                2.256294e+05
d                4.798951e+04
f                2.503322e+05
dtype: float64

In [230]:
optim_func([8.116929e-10, 2.256294e+05,  4.798951e+04, 2.503322e+05], OBSERVED_COUNTS)

16.688146814872198